In [1]:
import sqlite3
import requests
import string

In [2]:
conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')

In [3]:
c = conn.cursor()

In [4]:
c.execute("""SELECT *
          From sales
          Where id_sale Like '123456'""")
rows = c.fetchall()
rows

[(123456,
  384923,
  'CHIP37648',
  '2021-12-01 12:00:00-03:00',
  'in progress',
  'in the delibery step')]

# Functions needed

## Settings

In [5]:
#API root and auth
sale_id_api = 'https://logistics-api-dot-active-thunder-329100.rj.r.appspot.com'
chip_api = 'https://telecom-api-dot-active-thunder-329100.rj.r.appspot.com'
conversations_api = 'https://chats-api-dot-active-thunder-329100.rj.r.appspot.com'
header = {'authorization': 'teste'} #auth code

#HTTP Requests
#   Sale API
HTTP_request_sale_track = '/tracking'
HTTP_request_sale_zip = '/zip_code'
#   Chip Api
HTTP_request_chip = '/chip_status'
#   Conversation API
HTTP_request_conversation = '/conversations'
HTTP_request_conversation_info = '/conversation_info'

## Functions


### Tracking Function

In [6]:
def generate_sale_tracking_info(params_request):
    url = sale_id_api+HTTP_request_sale_track

    response_sale_track = requests.post(url, json=params_request, headers=header)
    return response_sale_track.json()

### Zip code function

In [223]:
def generate_sale_zip_info(params_request):
    url = sale_id_api+HTTP_request_sale_zip

    response_sale_zip = requests.post(url, json={'zip_code':params_request}, headers=header)
    return response_sale_zip.json()

### Chip Status function

In [8]:
def generate_chip_info(params_request):
    url = chip_api+HTTP_request_chip

    response_chip = requests.post(url, json=params_request, headers=header)
    return response_chip.json()

### Active conversation function

In [16]:
def generate_act_conv_info():
    response_conversation = requests.post(conversations_api+HTTP_request_conversation, headers=header)
    json_response_conversation = response_conversation.json() #retrieve active conversations ID

    active_chat_list = [] #Create a list of active conversations info
    for values in json_response_conversation.values():
        for val in values:
            params_conver_info = {'conversation_id': str(val)}
            response_conversation = requests.post(conversations_api+HTTP_request_conversation_info, headers=header, json=params_conver_info)
            active_chat_list.append(response_conversation.json())

        return active_chat_list

In [255]:
generate_act_conv_info()[0]

{'conversation_id': '754893',
 'messages': [{'message_id': '754893-0',
   'text': "hello, my money hasn't landed in my bank account yet. I want to know what happened",
   'created_at': '1640197668'}],
 'merchant_id': '558392',
 'subject': 'bank account receipt problem'}

# SQL data recopilation

## Greetings

In [14]:
#help with the first active conversation -> need to parse all the active chats and resolve one by one
def greetings():
    active_chats_list = generate_act_conv_info()
    first_active_chat = active_chats_list[0]
    return print(f"Greetings! I'm e-capi chat bot.\nI'm here to help you with your \033[1m{first_active_chat.get('subject')}\033[0m"
)

In [15]:
greetings()

Greetings! I'm e-capi chat bot.
I'm here to help you with your bank account receipt problem


## Problems classifier

In [67]:
delivery_problem = ['delivery', 'deliver', 'arrive']
receipt_problem = ['receipt']
connection_problem = ['connection', 'connect']
problem_classifier = [delivery_problem, receipt_problem, connection_problem]

In [145]:
problem_classifier_dict ={'delivery_problem': ['delivery', 'deliver', 'arrive'],
                        'receipt_problem': ['receipt'],
                        'connection_problem': ['connection', 'connect']}


## Analyzer of subject -> define where to search data

In [167]:
_subject_problem = ['bank', 'account', 'receipt', 'problem']
def match_key_list(dictionary, subject_problem):
    for word in subject_problem:
        problem = [key for key,val in dictionary.items() if any(word in sub_list for sub_list in val)]
        if len(problem) != 0:
            break
        else: continue
    return problem

In [168]:
match_key_list(problem_classifier_dict, _subject_problem)

['receipt_problem']

In [285]:
#take out the while and leave just a subject analyzer
def subject_analyzer():
    #analyze first chat /!\ TB automatize
    active_chats_list = generate_act_conv_info()
    first_active_chat = active_chats_list[0]

    subject_to_analyze = first_active_chat.get('subject')

    subject_to_analyze = subject_to_analyze.translate(str.maketrans("","", string.punctuation))
    list_words_to_analyze = subject_to_analyze.split() #Create a list of words to make a 'dumb' analyzer

    problem_type = match_key_list(problem_classifier_dict, subject_to_analyze)
        
    return problem_type[0]

In [286]:
subject_analyzer()

'delivery_problem'

In [ ]:
def subject_analyzer_v2():
    #analyze first chat /!\ TB automatize
    active_chats_list = generate_act_conv_info()
    first_active_chat = active_chats_list[0]

    messages_to_analyze = first_active_chat.get('messages')
    subject_to_analyze = first_active_chat.get('subject')

    messages_pipeline = len(messages_to_analyze)

    while messages_pipeline != 0:
       
        subject_to_analyze = subject_to_analyze.translate(str.maketrans("","", string.punctuation))
        list_words_to_analyze = subject_to_analyze.split() #Create a list of words to make a 'dumb' analyzer
        
        problem_type = matchkey_list(problem_classifier_dict, subject_to_analyze)


        messages_pipeline = messages_pipeline - 1 #Resolve problem and go to next message
        
        return problem_type[0]

## metadata SQL recopilator

In [287]:
generate_act_conv_info()

[{'conversation_id': '754893',
  'messages': [{'message_id': '754893-0',
    'text': "hello, my money hasn't landed in my bank account yet. I want to know what happened",
    'created_at': '1640197668'}],
  'merchant_id': '558392',
  'subject': 'bank account receipt problem'},
 {'conversation_id': '754894',
  'messages': [{'message_id': '754894-0',
    'text': "my chip is not connected to the internet. I've already tested my wifi and it's ok, the problem is on the chip itself",
    'created_at': '1640197670'}],
  'merchant_id': '530364',
  'subject': 'connection problem'},
 {'conversation_id': '754895',
  'messages': [{'message_id': '754895-0',
    'text': "where is my credit card machine? it hasn't arrived yet, the delivery is already late",
    'created_at': '1640197673'}],
  'merchant_id': '384923',
  'subject': 'late delivery'},
 {'conversation_id': '754896',
  'messages': [{'message_id': '754896-0',
    'text': "Hi! I wanted to know where my machine is. I'm going on a vacation nex

### tests

In [27]:
act_convers = generate_act_conv_info()
act_convers_pipe = len(act_convers) #needed for the while ?

for conversation in act_convers: #Salutation and state the problem
    print(f"Greetings! I'm e-capi chat bot.\nI'm here to help you with your \033[1m{conversation.get('subject')}\033[0m")
    
    messages_on_active_conver = len(conversation.get('messages'))
    while messages_on_active_conver !=0:
        
        
        messages_on_active_conver -= 1 #mark message as resolve\

Greetings! I'm e-capi chat bot.
I'm here to help you with your bank account receipt problem
Greetings! I'm e-capi chat bot.
I'm here to help you with your connection problem
Greetings! I'm e-capi chat bot.
I'm here to help you with your late delivery
Greetings! I'm e-capi chat bot.
I'm here to help you with your check delivery address
Greetings! I'm e-capi chat bot.
I'm here to help you with your delivery forecast


### Func testing

**Function below need a funct that retrieve the merchant_id and parse sql for info**
- This function resolve act conversations 1 by 1 and within messages 1 by 1

In [ ]:
def asw_act_conv_pipeline():
    act_convers = generate_act_conv_info()
    act_convers_pipe = len(act_convers) #needed for the while ?

    for conversation in act_convers: #Salutation and state the problem
        print(f"Greetings! I'm e-capi chat bot.\nI'm here to help you with your \033[1m{conversation.get('subject')}\033[0m")

        messages_on_active_conver = len(conversation.get('messages'))
        while messages_on_active_conver !=0:
            #funct that retrieve the merchant_id and parse sql for info
            

            #if delivery problem
            sale_id = get_sale_id_SQL(conversation)
            
            messages_on_active_conver -= 1 #mark message as resolve
    

## get sale_id from merchant id

In [208]:
#get for the first act conversation /!\ need to do it for all act convers
def get_sale_id_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT id_sale
        FROM sales
        WHERE sales.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    sale_id = c.fetchone()
    return {"id_sale": str(sale_id[0])} #gives a tupple, with a second null arg

In [209]:
get_sale_id_SQL(384923)

{'id_sale': '123456'}

## get receipt info from merchant_id

In [271]:
#get for the first act conversation /!\ need to do it for all act convers
def get_receipt_info_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT status, description, value
        FROM receipt
        WHERE receipt.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    receipt_info = c.fetchone()
    
    #variables
    status = receipt_info[0]
    description = receipt_info[1]
    value = receipt_info[2]
    
    answ_dict = {'status':status, 'description': description, 'value': value }
    return answ_dict
#     return {"status": str(status)} #gives a tupple, with a second null arg

In [272]:
get_receipt_info_SQL('558392')

{'status': 'failed',
 'description': "bank's API is unavailable",
 'value': 177.4}

## get info for the current problem

In [305]:
def answer_for_problem(conversation=generate_act_conv_info()):
    conversation = conversation[1] #TBDelt when implemented on asw_convers
    merchant_id = conversation.get('merchant_id')
    problem = 'receipt_problem'#subject_analyzer() #not working for others problem than delivery

    if problem == 'connection problem':
        
        #get info tracking API
        id_sale = get_sale_id_SQL(merchant_id)
        tracking_info = generate_sale_tracking_info(id_sale)
        
        #get info zip code API
        zip_code = tracking_info.get("destination_zip_code")
        zip_code_info = generate_sale_zip_info(zip_code)
        
        #variables
        delivery_forecast = tracking_info.get('delivery_forecast')
        order_status = tracking_info.get('status')
        neighborhood = zip_code_info.get('neighborhood')
        city = zip_code_info.get('city')
        street = zip_code_info.get('street')
        state = zip_code_info.get('state')
        
#         if len(zip_code_info.get('complement')) != 0: #do something when there is a complement
#             complement = zip_code_info.get('complement')
        
        return print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m.\nStatus: \033[1m{order_status}\033[0m")
        
        
    elif problem == 'receipt_problem':
        #get receipt info from SQL
        receipt_info = get_receipt_info_SQL(merchant_id)
        
        #variables
        status_receipt = receipt_info.get('status')
        description_receipt = receipt_info.get('description')
        value_receipt = receipt_info.get('value')
        
        print(f"\033[1mFirst of all we would like to remind you that transactions have a 24h delay before being transfered.\033[0m\nConcerning your demand here is some information:\nTotal Amount: \033[1mR${value_receipt}\033[0m\nStatus: \033[1m{status_receipt}\033[0m\nAdditional info:\033[1m{description_receipt}\033[0m")
    
    elif problem == 'connection_problem':
        pass
    
    
    else:
        pass
            

In [306]:
answer_for_problem()

First of all we would like to remind you that transactions have a 24h delay before being transfered.
Concerning your demand here is some information:
Total Amount: R$177.4
Status: failed
Additional info:bank's API is unavailable


In [308]:
generate_act_conv_info()[1]

{'conversation_id': '754894',
 'messages': [{'message_id': '754894-0',
   'text': "my chip is not connected to the internet. I've already tested my wifi and it's ok, the problem is on the chip itself",
   'created_at': '1640197670'}],
 'merchant_id': '530364',
 'subject': 'connection problem'}